### Simple examples of ensembling models

Expected that hyperparameters of base models are already tuned

In [1]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

In [3]:
SHIFT = 190
os.environ['LIGHTGBM_EXEC'] = '/home/eamag/apps/LightGBM'  # path to LightGBM executable

In [4]:
df = pd.read_csv('~/data/allstate/train.csv.zip', compression='zip')

for column in tqdm(df.columns):
    encoder = LabelEncoder()
    if column.startswith('cat') :
        df[column] = encoder.fit_transform(df[column])
        df['{}_sz'.format(column)] = df[column].map(df.groupby(column).size())

FileNotFoundError: [Errno 2] No such file or directory: '/home/eamag/data/allstate/train.csv.zip'

###### For speed and simplicity there are only one split 80% / 20%
This is equivalent to a single iteration of 5-fold CV

In [4]:
train, test = train_test_split(df, test_size = 0.2, random_state = 0)
y_train, y_test = train['loss'], test['loss']
del train['loss'], test['loss']

###### Fit simple XGBoost and LightGBM models (without any tuning)

In [5]:
%%time
dtrain = xgb.DMatrix(train, label=np.log(y_train + SHIFT))
dtest = xgb.DMatrix(test)
params = {'silent': 1}
model = xgb.train(params, dtrain, 100)
y_pred_1 = np.exp(model.predict(dtest)) - SHIFT
print(mean_absolute_error(y_pred_1, y_test))

1162.57690445
CPU times: user 4min 55s, sys: 1.35 s, total: 4min 56s
Wall time: 1min 24s


In [6]:
%%time
from pylightgbm.models import GBMRegressor
params = {'verbose': False, 'num_iterations': 100}
clf = GBMRegressor(**params)
clf.fit(train, np.log(y_train + SHIFT))
y_pred_2 = np.exp(clf.predict(test)) - SHIFT
print(mean_absolute_error(y_pred_2, y_test))

1150.66772131
CPU times: user 27 s, sys: 1.21 s, total: 28.2 s
Wall time: 1min 11s


###### Get mean of this predictions

In [7]:
y_pred = (y_pred_1 + y_pred_2)/2
print(mean_absolute_error(y_pred, y_test))

1147.67752573


###### Try to choose weights for convex combination of predictions
i.e the sum of the weights is equal to one

In [8]:
best_a = None
min_err = 10**6
for a in np.arange(.05, 1, .05):
    y_pred = a*y_pred_1 + (1 - a)*y_pred_2
    err = mean_absolute_error(y_pred, y_test)
    if err < min_err:
        min_err = err
        best_a = a
    print(a, err)
print('----------------')
print('best:', best_a, min_err)

0.05 1149.49742146
0.1 1148.50355639
0.15 1147.7101578
0.2 1147.12598108
0.25 1146.74645865
0.3 1146.56618344
0.35 1146.57364663
0.4 1146.76553844
0.45 1147.12663828
0.5 1147.67752573
0.55 1148.38881233
0.6 1149.24503975
0.65 1150.28968449
0.7 1151.50808495
0.75 1152.90743673
0.8 1154.48431484
0.85 1156.22900358
0.9 1158.16242752
0.95 1160.28705474
----------------
best: 0.3 1146.56618344


The result is better than simple mean.

When using such method better models will have larger weight. This can be a problem if you have models with different quality.

### Stacking

###### Get out of fold predictions

In [9]:
import numpy as np
import xgboost as xgb
from pylightgbm.models import GBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

N_SPLITS = 5

def get_xgb_pred(X_train, X_test, y_train, params=None, n_trees=100, res_transform=None):
    if params is None:
        params = {'silent': 1}
    if res_transform is None:
        res_transform = lambda x: x
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test)
    model = xgb.train(params, dtrain, n_trees)
    return res_transform(model.predict(dtest))

def get_lightgbm_pred(X_train, X_test, y_train, params=None, n_trees=100, res_transform=None):
    if params is None:
        params = {'verbose': False}
    if not 'num_iterations' in params:
        params['num_iterations'] = n_trees
    if res_transform is None:
        res_transform = lambda x: x
    clf = GBMRegressor(**params)
    clf.fit(X_train, y_train)
    return res_transform(clf.predict(X_test))

def predict_outoffolds(X, y, X_test=None, func='get_xgb_pred', params=None, res_transform=None, n_splits=3):
    if X_test is None:
        kf = KFold(n_splits=n_splits)
        y_pred = np.zeros_like(y.values)
        for train_inds, test_inds in kf.split(X):
            y_pred[test_inds] = globals()[func](X.iloc[train_inds], X.iloc[test_inds],
                                                y.iloc[train_inds], res_transform=res_transform, params=params)
        return y_pred
    else:
        return globals()[func](X, X_test, y)

In [10]:
train_stack = pd.DataFrame(index=train.index)
test_stack = pd.DataFrame(index=test.index)

In [11]:
%%time
train_stack['y_xgb'] = predict_outoffolds(train, np.log(y_train + SHIFT), func='get_xgb_pred')
test_stack['y_xgb'] = predict_outoffolds(train, np.log(y_train + SHIFT), X_test=test, func='get_xgb_pred')
train_stack['y_lightgbm'] = predict_outoffolds(train, np.log(y_train + SHIFT), func='get_lightgbm_pred')
test_stack['y_lightgbm'] = predict_outoffolds(train, np.log(y_train + SHIFT), X_test=test, func='get_lightgbm_pred')
train_stack['y_xgb'] = np.exp(train_stack.y_xgb) - SHIFT
test_stack['y_xgb'] = np.exp(test_stack.y_xgb) - SHIFT
train_stack['y_lightgbm'] = np.exp(train_stack.y_lightgbm) - SHIFT
test_stack['y_lightgbm'] = np.exp(test_stack.y_lightgbm) - SHIFT

CPU times: user 15min 28s, sys: 7.19 s, total: 15min 35s
Wall time: 7min 13s


In [12]:
print(mean_absolute_error(train_stack.y_xgb.fillna(0), y_train),
      mean_absolute_error(train_stack.y_lightgbm.fillna(0), y_train),
      mean_absolute_error(test_stack.y_xgb.fillna(0), y_test),
      mean_absolute_error(test_stack.y_lightgbm.fillna(0), y_test))

1168.72252642 1149.25187965 1162.57690445 1150.66772131


###### Fit linear regression without bias on this two features

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
lr = LinearRegression(fit_intercept=False)
lr.fit(train_stack[['y_xgb', 'y_lightgbm']], y_train)
y_pred = lr.predict(test_stack[['y_xgb', 'y_lightgbm']])
print(mean_absolute_error(y_pred, y_test))

1166.50388063


In [15]:
print(*lr.coef_)

0.420327063731 0.693043542054


###### Fit nonlinear model on second level
concatenate meta features with original features

In [16]:
_train = pd.concat([train, train_stack], axis=1)
_test = pd.concat([test, test_stack], axis=1)

In [17]:
%%time
from pylightgbm.models import GBMRegressor
params = {'verbose': False, 'num_iterations': 100}
clf = GBMRegressor(**params)
clf.fit(_train, np.log(y_train + SHIFT))
y_pred = np.exp(clf.predict(_test)) - SHIFT
print(mean_absolute_error(y_pred, y_test))

1145.20823757
CPU times: user 25.7 s, sys: 1.18 s, total: 26.9 s
Wall time: 1min 2s


That's better than convex combination of predictions

###### Try to add yet another LightGBM
parameters are intentionally set to non-optimal values to get a weaker model

In [18]:
%%time
params = {
    'num_iterations': 100,
    'num_leaves': 127,
    'feature_fraction': .5,
    'bagging_fraction': .5,
    'learning_rate': .06,
    'min_data_in_leaf': 10,
    'verbose': False
}
from pylightgbm.models import GBMRegressor
clf = GBMRegressor(**params)
clf.fit(train, np.log(y_train + SHIFT))
y_pred_3 = np.exp(clf.predict(test)) - SHIFT
print(mean_absolute_error(y_pred_3, y_test))

1164.09046584
CPU times: user 25.2 s, sys: 1.18 s, total: 26.3 s
Wall time: 54 s


In [19]:
y_pred = (y_pred_1 + y_pred_2 + y_pred_3)/3
print(mean_absolute_error(y_pred, y_test))

1148.42467693


weak model worsened the result of averaging

##### Convex combination of predictions tuned by Sequential Least Squares Programming solver
https://github.com/jdwittenauer/kaggle/blob/master/OttoGroup/find_ensemble_weights.py

http://stackoverflow.com/questions/35631192/element-wise-constraints-in-scipy-optimize-minimize

In [20]:
from scipy.optimize import minimize
def log_loss_func(weights):
    """
    scipy minimize will pass the weights as a numpy array
    """
    final_prediction = 0
    for weight, prediction in zip(weights, preds):
            final_prediction += weight*prediction
    return mean_absolute_error(y_test, final_prediction)

In [21]:
preds = [y_pred_1, y_pred_2, y_pred_3]

In [22]:
init_weights = np.ones(len(preds)) / len(preds)
# adding constraints
cons = ({'type': 'eq', 'fun': lambda w: 1-sum(w)})  # weights are sum to 1
bounds = [(0, 1)] * len(preds)  # and bounded between 0 and 1
w = minimize(log_loss_func, init_weights, method='SLSQP', bounds=bounds, constraints=cons)

In [23]:
w.x

array([ 0.30029172,  0.46381807,  0.23589021])

In [24]:
mean_absolute_error(np.dot(w.x.reshape(1, -1), np.vstack([y_pred_1, y_pred_2, y_pred_3])).ravel(), y_test)

1147.4365723451813

Slightly better than simple mean, but it is not the general case

###### Get out-of-fold prediction for this model

In [25]:
%%time
train_stack['y_lightgbm2'] = predict_outoffolds(train, np.log(y_train + SHIFT), func='get_lightgbm_pred', params=params)
test_stack['y_lightgbm2'] = predict_outoffolds(train, np.log(y_train + SHIFT), X_test=test, func='get_lightgbm_pred',
                                               params=params)
train_stack['y_lightgbm2'] = np.exp(train_stack.y_lightgbm2) - SHIFT
test_stack['y_lightgbm2'] = np.exp(test_stack.y_lightgbm2) - SHIFT

CPU times: user 1min 27s, sys: 4.3 s, total: 1min 31s
Wall time: 3min 8s


In [26]:
print(mean_absolute_error(train_stack.y_lightgbm2.fillna(0), y_train),
      mean_absolute_error(test_stack.y_lightgbm2.fillna(0), y_test))

1158.32348978 1150.66772131


###### Fit nonlinear model on second level
concatenate meta features with original features

In [27]:
_train = pd.concat([train, train_stack], axis=1)
_test = pd.concat([test, test_stack], axis=1)

In [28]:
%%time
params = {'verbose': False, 'num_iterations': 100, 'num_leaves': 63}
clf = GBMRegressor(**params)
clf.fit(_train, np.log(y_train + SHIFT), test_data=[(_test, np.log(y_test + SHIFT))])
y_pred = np.exp(clf.predict(_test)) - SHIFT
print(mean_absolute_error(y_pred, y_test))

1144.95280688
CPU times: user 31.5 s, sys: 1.46 s, total: 33 s
Wall time: 1min 3s


###### Some links
http://mlwave.com/kaggle-ensembling-guide/

https://www.kaggle.com/c/allstate-claims-severity/forums/t/25743/stacking-understanding-python-package-for-stacking

https://www.kaggle.com/c/allstate-claims-severity/forums/t/25240/weights-in-an-ensemble